In [1]:
#data prep + create fluxes + call trainer
import sys
sys.path.append('Glycolysis/')

from fluxes import create_fluxes
from glycolysis import *

import torch
#from torchvision import models
from torchsummary import summary
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchdiffeq import odeint_adjoint as odeint
from torch.autograd import Variable
from torch import optim



timeseries=pd.read_csv("data/FF1_timeseries_format.csv",index_col=0)
time_points=timeseries.columns.to_list()
time_points=[int(i) for i in time_points]

## metabolite names as implemented in the glycolysis model of Olga.
metabolites=['Glci','G6P','G1P','T6P',
 'TRE','F6P','F16BP','GAP',
 'DHAP','G3P','GLYCEROL','BPG',
 'P3G','P2G','PEP','PYR',
 'ACE','ETOH','ATP','ADP',
 'AMP','PI','IMP','INO',
 'HYP','NAD','NADH','Glco']

#initialization of missing species and adding to the timeseries (only initial value)
timeseries.loc['IC_BPG','0']=0.0
timeseries.loc['IC_ACE','0']=0.0400
timeseries.loc['EC_ETOH','0']=0.000
timeseries.loc['IC_PI','0']=10
timeseries.loc['IC_IMP','0']=0.1000
timeseries.loc['IC_INO','0']=0.1000
timeseries.loc['IC_HYP','0']=1.5000
timeseries.loc['IC_NAD','0']=1.5794
timeseries.loc['IC_NADH','0']=0.0106

timeseries.loc['ICglyc','0']=0.1000
timeseries.loc['EC_Glucose',:]=0.18
# timeseries.loc['EC_Glucose',['10','15']]=110,55.09
#timeseries.loc['EC_glucose','0']=0.2
#timeseries.loc['IC_UDPG','0']=0.07

index=[2,8,7,5,
3,9,10,11,
12,30,4,34,
13,14,15,16,
35,36,31,32,
33,37,38,39,
40,41,42,43] #Indices in the time series belonging to the metabolites

#subset the metabolites from dataframe
timeseries=timeseries.iloc[index,:]
#Fix this confusing naming
timeseries.index=metabolites

metabolites_dictionary=dict(zip(timeseries.index.to_list(),np.arange(0,np.shape(timeseries)[0]))) 

fluxes=create_fluxes()
metabolites=metabolites_dictionary #dictionary that binds indices in timeseries data to metabolites in glycolysis model
glycolysis=Glycolysis(fluxes=fluxes, metabolites=metabolites)


tensor_c0=torch.tensor(timeseries['0'])

# predicted_c = odeint(func=glycolysis, y0=tensor_c0, t=torch.Tensor(time_points),method="euler")
# predicted_c = odeint(func=glycolysis, y0=tensor_c0, t=torch.Tensor(time_points),method="scipy_solver",options={"solver":"LSODA"},atol=1e-6,rtol=1e-3)
# print(np.shape(predicted_c))







In [2]:

predicted_c = odeint(func=glycolysis, y0=tensor_c0, t=torch.Tensor(time_points),method="euler")#method="scipy_solver",options={"solver":"BDF"},atol=1e-6,rtol=1e-3)
predicted_c

UnboundLocalError: local variable 'GLCo' referenced before assignment

In [12]:
fluxes=create_fluxes()
metabolites=metabolites_dictionary #dictionary that binds indices in timeseries data to metabolites in glycolysis model

#ODE system
glycolysis=Glycolysis(fluxes=fluxes, metabolites=metabolites)

#Concentrations
tensor_concentrations=torch.tensor(np.array(timeseries.T),dtype=torch.float64,requires_grad=True)

#Timepoint to evaluate
tensor_timepoints=torch.tensor(time_points,dtype=torch.float64,requires_grad=False)
rate_coeff_params=list(glycolysis.parameters()) # for a regularization term

optimizer=optim.AdamW(glycolysis.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=20)

device="cpu"
tensor_concentrations = tensor_concentrations.to(device)
tensor_timepoints = tensor_timepoints.to(device)

def loss_func():
    # ls=0
    try:
        tensor_c0=tensor_concentrations[0,:]

        target=tensor_concentrations
        predicted_c = odeint(func=glycolysis, y0=tensor_c0, t=torch.Tensor(time_points),method="dopri5")#,options={"solver":"dopri5"},atol=1e-6,rtol=1e-3)
        #first filter out any value that is nan in the target time series data 
        boolean_mask=~target.isnan() 
        target=torch.masked_select(target,boolean_mask)
        predicted_c=torch.masked_select(predicted_c,boolean_mask)

        #then do the same thing for everything that is nan in the prediction 
        boolean_mask=~predicted_c.isnan() 
        
        predicted_c=torch.masked_select(predicted_c,boolean_mask)
        target=torch.masked_select(target,boolean_mask)
        
        # print(target.requires_grad,predicted_c.requires_grad)
        ls =torch.mean(torch.square((predicted_c - target)))

        # ls=torch.Tensor(ls, requires_grad = False)
    except:
        print("do this")
    return ls


# optimizer=optim.AdamW(glycolysis.parameters(), lr=1e-2)

# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=20)

get_loss_per_iteration=[]
with torch.enable_grad():
    for i in range(0,200,1):
        scheduler_step=True
        optimizer.zero_grad()
        loss = loss_func()
        
        print(loss.requires_grad)
        
        get_loss_per_iteration.append(loss.detach().numpy())
        # print(loss.requires_grad)

        loss.backward()
        optimizer.step()
        
    #     # print(loss.backward())
        if scheduler_step:
            scheduler.step(loss)


# # optimizer.zero_grad()

do this


UnboundLocalError: local variable 'ls' referenced before assignment

In [84]:
sys.path.append('IRDM/')
from interpolated_torchdiffeq import odeint_chebyshev_func
from functools import partial 
n_nodes = 10  # if you want 10 grid points
odeint_chebyshev = partial(odeint_chebyshev_func, n_nodes=n_nodes)
help(odeint_chebyshev_func)
T=torch.Tensor([0.0,0.001])


predicted_c=odeint_chebyshev_func(y0=tensor_c0,func=glycolysis,t=T,rtol=1e-3,atol=1e-6,method="euler",n_nodes=10)

np.shape(predicted_c.detach().numpy())

Help on function odeint_chebyshev_func in module interpolated_torchdiffeq._impl.odeint_interpolated:

odeint_chebyshev_func(func, y0, t, rtol=1e-06, atol=1e-12, method=None, options=None, n_nodes=None, return_intermediate_points=False)



(2, 28)